# **Project Akhir Jaringan Saraf Tiruan**
## Anggota Kelompok
- Vincentia Melody Vivianne (235150201111047)
- Ana Zahratul Firdausi (235150201111049)
- Rizkyka Atila Zakiya (235150201111050)




### Tahap 1: Load Dataset

In [ ]:
from google.colab import files
import io, csv, random, math, copy, sys
from collections import defaultdict
random.seed(42)

print("Upload CSV dataset (convert from .xlsx to .csv if perlu).")
uploaded = files.upload()
file_name = list(uploaded.keys())[0]
print("Loaded file:", file_name)

data = []
with io.TextIOWrapper(io.BytesIO(uploaded[file_name]), encoding='utf-8') as f:
    reader = csv.reader(f)
    for row in reader:
        if not any(cell.strip() for cell in row):
            continue
        data.append(row)

if len(data) < 2:
    raise ValueError("Dataset terlalu kecil atau format tidak sesuai.")

print("Contoh baris awal:")
for r in data[:5]:
    print(r)

Upload CSV dataset (convert from .xlsx to .csv if perlu).


Saving data2.csv to data2 (14).csv
Loaded file: data2 (14).csv
Contoh baris awal:
['Widht', 'Length', 'Diameter', 'Perimeter', 'Area', 'AspectRatio', 'FormFactor', 'Rectangularity', 'Narrow Factor', 'RatioOfDiameter', 'RatioPLPW', 'Solidity', 'Convexity', 'Class']
['192', '698', '325,7666027', '1645,099629', '83349,5', '3,635416667', '0,3870155846', '1,607880071', '0,4667143305', '5,049933344', '1,84842655', '0,8446657039', '0,935651462', '1']
['206', '665', '317,5811088', '1589,643785', '79213,5', '3,22815534', '0,3939212721', '1,729376937', '0,4775655772', '5,005473377', '1,825078973', '0,8086104377', '0,9401741181', '1']
['269', '527', '314,285503', '1519,616436', '77578', '1,959107807', '0,4221633029', '1,82736085', '0,5963671784', '4,835146454', '1,909065874', '0,688771003', '0,876343809', '1']
['259', '553', '318,5028846', '1504,503738', '79674', '2,135135135', '0,4423234283', '1,797662977', '0,5759545833', '4,723673822', '1,852837116', '0,7160098854', '0,9023514967', '1']


### Tahap 2: Ekstraksi Fitur dan Label



In [ ]:
header = data[0]
rows = data[1:]

X_raw = []
y_raw = []

for r in rows:
    clean_row = [c.strip().replace(',', '.') for c in r]
    feats = []
    for v in clean_row[:-1]:
        if v == '' or v is None:
            feats.append(0.0)
            continue
        try:
            feats.append(float(v))
        except:
            feats.append(0.0)

    lbl_str = clean_row[-1]
    if lbl_str == '' or lbl_str is None:
        lbl = 0
    else:
        try:
            lbl = int(float(lbl_str))
        except:
            try:
                lbl = int(lbl_str)
            except:
                lbl = abs(hash(lbl_str)) % 1000

    X_raw.append(feats)
    y_raw.append(lbl)

n_samples = len(X_raw)
n_features = len(X_raw[0])
print(f"Samples: {n_samples}, Features: {n_features}")

Samples: 5656, Features: 13


### Tahap 3: Normalisasi Data (Min-Max Scaling)

In [ ]:
mins = [min(col) for col in zip(*X_raw)]
maxs = [max(col) for col in zip(*X_raw)]

denoms = []
for mi, ma in zip(mins, maxs):
    d = ma - mi
    if d == 0:
        d = 1.0
    denoms.append(d)

X_norm = []
for row in X_raw:
    new = []
    for i, v in enumerate(row):
        nv = (v - mins[i]) / denoms[i]
        if math.isnan(nv) or math.isinf(nv):
            nv = 0.0
        new.append(nv)
    X_norm.append(new)

X = X_norm
y = y_raw

### Tahap 4: Pembagian Data Train & Test

In [ ]:
by_class = defaultdict(list)
for idx, lbl in enumerate(y):
    by_class[lbl].append(idx)

train_idx = []
test_idx = []
train_ratio = 0.8
for lbl, indices in by_class.items():
    random.shuffle(indices)
    cut = int(len(indices) * train_ratio)
    train_idx.extend(indices[:cut])
    test_idx.extend(indices[cut:])

random.shuffle(train_idx)
random.shuffle(test_idx)

X_train = [X[i] for i in train_idx]
y_train = [y[i] for i in train_idx]
X_test  = [X[i] for i in test_idx]
y_test  = [y[i] for i in test_idx]

print("Train samples:", len(X_train), "Test samples:", len(X_test))
classes = sorted(list(set(y_train)))
print("Detected classes (train):", classes)

Train samples: 4522 Test samples: 1134
Detected classes (train): [1, 2, 3, 4, 5, 6]


### Tahap 5: Inisialisasi Prototipe (Codebook Vector)

In [ ]:
K = 30
prototypes = {}

for c in classes:
    candidates = [X_train[i] for i in range(len(X_train)) if y_train[i] == c]
    if len(candidates) == 0:
        raise ValueError(f"No training samples for class {c}")
    if len(candidates) >= K:
        chosen = random.sample(candidates, K)
    else:
        chosen = [random.choice(candidates) for _ in range(K)]
    prototypes[c] = [list(p) for p in chosen]

total_protos = sum(len(v) for v in prototypes.values())
print("Prototypes per class:", {c: len(prototypes[c]) for c in prototypes}, " total:", total_protos)

Prototypes per class: {1: 30, 2: 30, 3: 30, 4: 30, 5: 30, 6: 30}  total: 180


### Tahap 6: Perhitungan Jarak Euclidean

In [ ]:
def euclid(a, b):
    s = 0.0
    for i in range(len(a)):
        diff = a[i] - b[i]
        s += diff * diff
    if s < 0:
        s = 0.0
    return math.sqrt(s)

## Tunning Parameter


Parameter 1: (lrate0=0.01 ;Epoch=100)

Parameter 2: (lrate0=0.02 ;Epoch=100)

Parameter 3: (lrate0=0.03 ;Epoch=100)

Parameter 4: (lrate0=0.05 ;Epoch=100)

Parameter 5: (lrate0=0.03 ;Epoch=150)

Parameter 6: (lrate0=0.05 ;Epoch=150)

Parameter 7: (lrate0=0.1 ;Epoch=100)

### Tahap 7: Implementasi Update LVQ 1


In [ ]:
def lvq1_update(x, true_label, prototypes, lrate=0.05):
    best_d = float('inf')
    best_cls = None
    best_i = None

    for cls in prototypes:
        for idx_p, p in enumerate(prototypes[cls]):
            d = euclid(x, p)
            if d < best_d:
                best_d = d
                best_cls = cls
                best_i = idx_p

    if best_cls == true_label:
        for j in range(len(x)):
            prototypes[best_cls][best_i][j] += lrate * (x[j] - prototypes[best_cls][best_i][j])
    else:
        for j in range(len(x)):
            prototypes[best_cls][best_i][j] -= lrate * (x[j] - prototypes[best_cls][best_i][j])

    for cls in prototypes:
        for idx_p in range(len(prototypes[cls])):
            for j in range(len(prototypes[cls][idx_p])):
                v = prototypes[cls][idx_p][j]
                if math.isnan(v) or math.isinf(v):
                    v = 0.0
                if v < 0.0:
                    v = 0.0
                if v > 1.0:
                    v = 1.0
                prototypes[cls][idx_p][j] = v

### Tahap 8: Training Model LVQ 1

In [ ]:
def evaluate(prototypes, Xs, ys):
    correct = 0
    conf = {}
    for x, t in zip(Xs, ys):
        best_d = float('inf')
        best_cls = None
        for cls in prototypes:
            for p in prototypes[cls]:
                d = euclid(x, p)
                if d < best_d:
                    best_d = d
                    best_cls = cls
        if best_cls == t:
            correct += 1
        conf[(t, best_cls)] = conf.get((t, best_cls), 0) + 1
    acc = correct / len(Xs) if len(Xs) > 0 else 0.0
    return acc, conf

lrate0 = 0.05
epochs = 150
min_lrate = 1e-6
print("Training LVQ-1 with lrate0=", lrate0, "epochs=", epochs)

for ep in range(epochs):
    current_lrate = lrate0 * (1 - (ep / epochs))

    if current_lrate < min_lrate:
        current_lrate = min_lrate
    pairs = list(zip(X_train, y_train))
    random.shuffle(pairs)

    for x, label in pairs:
        lvq1_update(x, label, prototypes, lrate=current_lrate)

    if (ep+1) % 20 == 0 or ep == 0 or ep == epochs-1:
        tr_acc, _ = evaluate(prototypes, X_train, y_train)
        te_acc, _ = evaluate(prototypes, X_test, y_test)
        print(f"Epoch {ep+1}/{epochs}  train_acc={tr_acc:.4f}  test_acc={te_acc:.4f}  lrate={current_lrate:.6f}")

print("Training selesai.")

Training LVQ-1 with lrate0= 0.05 epochs= 150
Epoch 1/150  train_acc=0.5117  test_acc=0.4224  lrate=0.050000
Epoch 20/150  train_acc=0.5478  test_acc=0.4903  lrate=0.043667
Epoch 40/150  train_acc=0.4569  test_acc=0.4145  lrate=0.037000
Epoch 60/150  train_acc=0.3050  test_acc=0.2972  lrate=0.030333
Epoch 80/150  train_acc=0.1659  test_acc=0.1640  lrate=0.023667
Epoch 100/150  train_acc=0.3180  test_acc=0.3183  lrate=0.017000
Epoch 120/150  train_acc=0.2477  test_acc=0.2593  lrate=0.010333
Epoch 140/150  train_acc=0.3262  test_acc=0.3307  lrate=0.003667
Epoch 150/150  train_acc=0.3514  test_acc=0.3616  lrate=0.000333
Training selesai.


### Tahap 9: Proses Testing

In [ ]:
test_acc, conf = evaluate(prototypes, X_test, y_test)
print(f"\nAccuracy: {test_acc:.4f}\n")

true_labels = sorted(list(set(y_test)))
pred_labels = sorted(list(set([pred for (_, pred) in conf.keys()])))
labels_all = sorted(list(set(true_labels + pred_labels)))

print("Confusion Matrix (true -> pred):")
print("     ", end="")
for p in labels_all:
    print(f"{p:6}", end="")
print()

for t in labels_all:
    print(f"{t:4} ", end="")
    for p in labels_all:
        print(f"{conf.get((t,p),0):6}", end="")
    print()


Accuracy: 0.3616

Confusion Matrix (true -> pred):
          1     2     3     4     5     6
   1    149     1     4     0    66     0
   2    102     3     4     3   106     0
   3      4     0    48    57    80     0
   4      2     0    16    83    91     0
   5     10     0     5    10   113     7
   6     15     1     0     9   131    14


##